In [1]:
from bn256 import *
import random
from copy import copy
import time

In [2]:
def timer(func):
    def _func(*args, **kwargs):
        t = time.clock()
        res = func(*args, **kwargs)
        t = time.clock() - t
        print('{0}: time elapsed {1:.8f} s.' .format(func.__name__, t))
        return res

    return _func

In [3]:
def hamminge_distance(s1, s2):
    l = len(s1)
    assert l == len(s2), "Undefined for sequences of unequal length."
    return sum(el1 != el2 for el1, el2 in zip(s1, s2)) / l

In [4]:
a = (-1, 1)
def gen_vector(size):
    return tuple(random.choice(a) for i in range(size))

In [5]:
v1, v2 = gen_vector(16), gen_vector(16)

In [6]:
print(v1, v2, sep="\n")

(1, 1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, -1, 1, -1, 1)
(-1, -1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, 1, 1, -1, 1)


In [7]:
def inverse_G1(curve):
    inv = copy(curve)
    inv.y = inv.y.additive_inverse()
    return inv

In [8]:
def inverse_G2(twist):
    inv = copy(twist)
    inv.negate()
    return inv

In [9]:
def transfer_vec_to_G1(vec):
    return tuple(inverse_G1(curve_G) if p == -1 else curve_G for p in vec)

In [10]:
def transfer_vec_to_G2(vec):
    return tuple(inverse_G2(twist_G) if p == -1 else twist_G for p in vec)

In [11]:
def gen_keys(size):
    return tuple(random.randrange(2, order) for i in range(size))

In [12]:
v1points = transfer_vec_to_G2(v1)
v2points = transfer_vec_to_G1(v2)

In [13]:
v1points

(((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (20666913350058776956210519119118544732556678129809273996262322366050359951122,17778617556404439934652658462602675281523610326338642107814333856843981424549)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (20666913350058776956210519119118544732556678129809273996262322366050359951122,17778617556404439934652658462602675281523610326338642107814333856843981424549)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (44333636345587826776585919623241361010268679977813729575614822660813824120661,47221932139242163798143780279757230461301747781284361464062811170020202647234)),
 ((2116796163654258025501177006657054

In [14]:
v2points

((1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781))

In [15]:
h1, h2 = random.randrange(2, order), random.randrange(2, order)
H1, H2 = curve_G.scalar_mul(h1), twist_G.scalar_mul(h2)

In [16]:
h1, h2

(3048102763084540357508419487033567904270787458292096615694249837332549662690,
 745426016090743547500138238516775121723965324951969076899107015692367326997)

In [17]:
@timer
def master_key_generation():
    s, t = gen_keys(16), gen_keys(16)
    u, v = gen_keys(18), gen_keys(18)
    gen1_h = tuple(point_add(curve_G.scalar_mul(si), H1.scalar_mul(ti)) for si, ti in zip(s, t))
    gen2_h = tuple(point_add(twist_G.scalar_mul(ui), H2.scalar_mul(vi)) for ui, vi in zip(u, v))
    return gen1_h, s, t, gen2_h, u, v

In [18]:
gen1_h, s, t, gen2_h, u, v = master_key_generation()

master_key_generation: time elapsed 13.03568176 s.


In [19]:
gen1_h, s, t, gen2_h, u, v

(((8701826452392652067791917181628722821243250014955194794490525160715570495678, 14599985252944145177020864159998658059510102114672134819362587522344472325821),
  (56259596455487375104956378908208652695657046604897063725129731180607399352826, 50509759486268442525847747939343481536054774550806762333649611203189408215673),
  (20347332259418888216363172289761077605586899153982130340000000922997510253474, 49514629267658917051028024981148405069504821206620336070875300635940337977884),
  (56001511295517309294313907658443054555465494969111827051592142509736071371959, 52511182198875294011950632477643179904832994687558659010149557718090012982941),
  (44514599271625648215605794944980323293256466243801526580181468244496005515673, 30947578626254312866636929073058560491962431078153859323629770446914030953644),
  (51440855154078145388377334500916772944729412091569000619079324254410669882432, 59599605181886998901076508168061295497897621662271004361717556000701598960870),
  (43144079111422467027249187

In [20]:
@timer
def decryption_key_generation(gen2_h, s, t, v1points):
    n = len(v1points)
    
    r0 = random.randrange(2, order)
    reg_template = [twist_G.scalar_mul(r0), H2.scalar_mul(r0)]
    
    d03 = v1points[0].scalar_mul(s[0])
    d04 = v1points[0].scalar_mul(t[0])
    for i in range(1, n):
        d03 = point_add(d03, v1points[i].scalar_mul(s[i]))
        d04 = point_add(d04, v1points[i].scalar_mul(t[i]))
    reg_template.append(point_add(inverse_G2(d03), gen2_h[0].scalar_mul(r0)))
    reg_template.append(point_add(inverse_G2(d04), gen2_h[1].scalar_mul(r0)))

    for i in range(n):
        reg_template.append(point_add(v1points[i], gen2_h[i + 2].scalar_mul(r0)))
    
    return tuple(reg_template)

In [21]:
DK = decryption_key_generation(gen2_h, s, t, v1points)

decryption_key_generation: time elapsed 15.14188925 s.


In [22]:
len(DK)

20

In [23]:
DK

(((11224927868596951531620996029649669876193725385428937067334221349209053038947,63990672280909742622950013346444888673912435808043552985455002869770818178220), (14650364845301937946771484213423329226901160947558616617705287946305311431669,46980420457459838720037316526845952796217294479834220649787397483096152158482)),
 ((17320692230495013850470230195788796497655289516991946253011328467051045172778,57491265757344952958618618546183244753894783460379696250777545927355793155814), (49107573418052273166611766364603238952027513575597904281088134149226057421220,8479293195121357583227562887657058473709590769314806835772486898404655325363)),
 ((45829756134562109515660961911037499711273802501068137438776614369710888470085,19574535427320584038501340151767500669445740773196400233873164571785922210116), (62675974920975275745989149406226125515687634817717626315524555926052645501122,56661250541771567801086717505475769186653405344305125010497978541543065586996)),
 ((54271064601621189825410315128137848

In [24]:
@timer
def encryption_authentication(gen1_h, u, v, v2points):
    n = len(v1points)
    
    r0 = random.randrange(2, order)
    auth_template = [curve_G.scalar_mul(0), curve_G.scalar_mul(0), curve_G.scalar_mul(r0), H1.scalar_mul(r0)]
    
    for ci, h in zip(v2points, gen1_h):
        auth_template.append(point_add(ci, h.scalar_mul(r0)))
        
    c01 = auth_template[2].scalar_mul(u[0])
    c02 = auth_template[2].scalar_mul(v[0])
    for i in range(1, n + 2):
        c01 = point_add(c01, auth_template[i + 2].scalar_mul(u[i]))
        c02 = point_add(c02, auth_template[i + 2].scalar_mul(v[i]))
    
    auth_template[0] = point_add(auth_template[0], inverse_G1(c01))
    auth_template[1] = point_add(auth_template[1], inverse_G1(c02))
    
    return tuple(auth_template)

In [25]:
CT = encryption_authentication(gen1_h, u, v, v2points)

encryption_authentication: time elapsed 3.97615797 s.


In [26]:
len(CT)

20

In [27]:
CT

((35689379534296819777687520302417889862309882870771820063646198340272302514849, 39577255693584163443727962047083179945818785044611447360148274500994721865115),
 (53688686584426523048649150596801413180479290947782972878867495338607552018234, 731456091568086414165530979942613169920992094409786773361430816806683400160),
 (26441880118820153275154742820848989650678538332150754591258567100636181481293, 35172627491403340054459904436632411793938212548427941734631648022562931536909),
 (57063108104053849871441853915437140451114732690683469515293331886665892968683, 58007223734792151355322468505212180648056153500331316708289380217626927917439),
 (20095794811698967044993577968942491753366534965481448479222618903906813725111, 31287569568897875582134254674780814990622278679179901848000963883769338201882),
 (18250789752238554443393658187828828844589036428876772597186449975387620350825, 34466413575282867310824800116441236308767334051418646943038114710327519373370),
 (6225674515945350118745844786058168

In [28]:
gt = optimal_ate(twist_G, curve_G)
logarithmic_table = [gt.exp(i) for i in range(16, -1, -2)]
gtm = optimal_ate(twist_G, inverse_G1(curve_G))
logarithmic_table.extend(gtm.exp(i) for i in range(2, 17, 2))

In [29]:
for i, el in enumerate(logarithmic_table):
    print(i, ": ", el)

0 :  (((15429308596100125345042025182838740585933899260990097211943232471185708209892,21036769640615344552925704451163498369215862996620789540674973406371475898801),(34643419400173764267828846220737244479195263121357284576095582127626220190690,39217953906512156784965537049391267021797863115575109425638059426537342418239),(29362508885087302295248031767237294697462853748363034881398247042502825752042,33065606296478674290051087119664030123566328954069047137872363079287669349803)),((44150688783494038190860128993786071429009665932812059212008957591291015396555,25855485444396824150330723346354416914393823131049522641950650987664836921571),(14355433543029713763013415741005074744216778912056694539583678910055649667776,46280493583452157638432486289407345007584489845082308921018418732775447515998),(35438949274284069241468591891906709017152328378869050685915198888251448805767,45559280046590109824720435394540600218774466880098159450808971756061223256757)))
1 :  (((597783055027885551703832584052683

In [30]:
@timer
def decryption_authentication(DK, CT):
    d = optimal_ate(DK[0], CT[0])
    for i in range(1, 20):
        d = d.mul(optimal_ate(DK[i], CT[i]))
    return logarithmic_table.index(d)

In [31]:
a = decryption_authentication(DK, CT)

decryption_authentication: time elapsed 15.54275238 s.


In [32]:
a

7

In [33]:
a / 16

0.4375

In [34]:
hamminge_distance(v1, v2)

0.4375